In [31]:
## Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.utils import class_weight
from collections import Counter
import torch
import numpy as np
import evaluate

## Load the dataset

In [32]:
# === Load CSV ===
dataset = load_dataset("csv", data_files="./AI_Human.csv")

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized = dataset.map(tokenize_fn, batched=True)
tokenized = tokenized.rename_column("generated", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dayso\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/487235 [00:00<?, ? examples/s]

In [33]:
# === Compute class weights ===
labels = dataset["train"]["generated"]
classes = np.unique(labels)
weights = class_weight.compute_class_weight("balanced", classes=classes, y=labels)
class_weights = torch.tensor(weights, dtype=torch.float)

print("Class distribution:", Counter(labels))
print("Class weights:", class_weights)

Class distribution: Counter({0.0: 305797, 1.0: 181438})
Class weights: tensor([0.7967, 1.3427])


In [35]:
# === Load RoBERTa-large with LoRA ===
base_model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

model = get_peft_model(base_model, lora_config)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# === Custom loss with class weights ===
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
    loss = loss_fn(logits, labels)
    return (loss, outputs) if return_outputs else loss

# === Evaluation metrics ===
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }


In [37]:
# === Training arguments ===
training_args = TrainingArguments(
    output_dir="./roberta-base-peft-ai-human",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    load_best_model_at_end=True
)

In [39]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").long()  
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [40]:
# === Split train/validation ===
split = tokenized["train"].train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
eval_ds = split["test"]


trainer = CustomTrainer(  # use custom trainer defined below
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,  # FutureWarning: will be removed — still works for now
    compute_metrics=compute_metrics
)

C:\Users\Dayso\AppData\Local\Temp\ipykernel_15960\3427178716.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(  # use custom trainer defined below
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [41]:
# === Train ===
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.068000,0.067489,0.986413,0.982170
2,0.000100,0.130544,0.978635,0.972310
3,0.000000,0.092069,0.984607,0.979898


c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b3a3fba0-e402-4795-a191-227b3aaccf1e)') - silently ignoring the lookup for the file config.json in roberta-base.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\save_and_load.py:238: UserWarning: Could not find a config file in roberta-base - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\Dayso\anaconda3\envs\pytorch_CV\Lib\site-packages\peft\utils\other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2d054ad0-9fc5-4217-a698-f58359c25a93)') - silently ignoring the lookup for the file config.json i

TrainOutput(global_step=164442, training_loss=0.0185511457487792, metrics={'train_runtime': 13902.298, 'train_samples_per_second': 94.627, 'train_steps_per_second': 11.828, 'total_flos': 1.7485804711937434e+17, 'train_loss': 0.0185511457487792, 'epoch': 3.0})